# Overview
Goal: make bentmark using liner regression

How:

results:

# Preprocess the data
+ fill na with average
+ encode categorical data not one-hot encoding

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [2]:
num_cols = train.columns.tolist()[1:-1]
cat_cols = 'EJ'

In [3]:
# fill nan to average
train = train.fillna(train.mean())

/tmp/ipykernel_20/131696701.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train = train.fillna(train.mean())


In [4]:
# label encoding
# categorical data to encoded data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# fit メソッドでラベルを覚えさせる
le.fit(train["EJ"])
# transform メソッドで数値に変換する
train["EJ"] = le.transform(train["EJ"])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# X and y
X = train[num_cols]
y = train['Class']

# Build the model

In [6]:
# モデルの作成
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1500)

# モデルの学習
model.fit(X, y)

# モデルの予測
y_pred = model.predict(X)

# モデルの評価
score = model.score(X, y)
print(score) # 0.9733333333333334

0.9011345218800648


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Output the prediction results

In [7]:
# X
test = test[num_cols]

In [8]:
test['EJ'] = le.transform(test['EJ'])

In [9]:
test["EJ"]

0    0
1    0
2    0
3    0
4    0
Name: EJ, dtype: int64

In [10]:
# predict data
# モデルの予測
y_pred = model.predict(test)
print(y_pred)

[0 0 0 0 0]


In [11]:
# モデルの予測
y_prob = model.predict_proba(test)

# 確率の表示
print(y_prob)

[[0.50005067 0.49994933]
 [0.50005067 0.49994933]
 [0.50005067 0.49994933]
 [0.50005067 0.49994933]
 [0.50005067 0.49994933]]


In [12]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    
    return logloss

In [13]:
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample_submission[['class_0','class_1']] = y_prob
sample_submission.to_csv('submission.csv',index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.500051,0.499949
1,010ebe33f668,0.500051,0.499949
2,02fa521e1838,0.500051,0.499949
3,040e15f562a2,0.500051,0.499949
4,046e85c7cc7f,0.500051,0.499949
